In [1]:
import os
import glob
import json
import subprocess
import soundfile as sf
import argparse

import pandas as pd
import numpy as np
import librosa
import scipy.io.wavfile
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from functools import partial

from sklearn.model_selection import StratifiedKFold, train_test_split

import tensorflow as tf
import tensorflow_io as tfio

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

main_dir = r"Datasets\TIMIT-dataset\data"

# Preprocessing

In [2]:
# wav_paths = [path for path in glob.glob(main_dir + "/*/*/*.wav") 
#     if not (path.endswith("_1.wav") or path.endswith("_2.wav"))]
# frames = [len(librosa.load(wav_path, sr=16000)[0]) for wav_path in tqdm(wav_paths)]
# print("Max frames:", np.max(frames))

# plt.figure(figsize=(10,4))
# sns.histplot(data=frames)
# plt.axvline(x=np.mean(frames), color='red')
# plt.xlabel("Number of frames per sample")
# plt.show()

## Split

In [3]:
# cutoff_limit = 70000
# outlier, split_files, frames = [], [], []
# for wav_path in tqdm(wav_paths, desc='Split'):
#     y = librosa.load(wav_path, sr=16000)[0]
#     n_frames = len(y)
#     frames.append(n_frames)
#     if n_frames >= cutoff_limit:
#         phn_path = wav_path.replace("wav", "phn")
#         lines = open(phn_path, "r").readlines()
#         end = np.array([int(line.split()[1]) for line in lines])
#         try: 
#             idx = np.where(end < n_frames//2)[0][-1]
#             lines_1 = lines[:idx]
#             lines_2 = lines[idx:]
#             factor = int(lines_2[0].split()[0])
#             for i, line in enumerate(lines_2):
#                 start, end, phoneme = line.split()
#                 start = int(start) - factor
#                 end = int(end) - factor
#                 line = " ".join([str(start), str(end), phoneme + "\n"])
#                 lines_2[i] = line
#             f1_path = os.path.splitext(phn_path)[0] + "_1.phn"
#             f2_path = os.path.splitext(phn_path)[0] + "_2.phn"
#             for path, lines in zip([f1_path, f2_path], [lines_1, lines_2]):
#                 with open(path, "w") as f:
#                     f.writelines(lines)    
#             sf.write(f1_path.replace("phn", "wav"), y[:factor], 16000)
#             sf.write(f2_path.replace("phn", "wav"), y[factor:], 16000)
#             split_files.append(wav_path)
#         except:
#             outlier.append(wav_path)
#             pass

# print("Outliers:\n", outlier)

In [4]:
# wav_paths = glob.glob(main_dir + "/*/*/*.wav")
# wav_paths = [path for path in wav_paths if not path in (split_files + outlier)]
# frames = []
# for wav_path in wav_paths:
#     y = librosa.load(wav_path, sr=16000)[0]
#     n_frames = len(y)
#     frames.append(n_frames)

# plt.figure(figsize=(10,4))
# sns.histplot(data=frames)
# plt.axvline(x=np.mean(frames), color='red')
# plt.xlabel("Number of frames per sample")
# plt.show()

## Padding

In [5]:
# wav_paths = glob.glob(main_dir + "/*/*/*.wav")
# wav_paths = [path for path in wav_paths 
#     if not path in (split_files + outlier)]
    
# for wav_path in tqdm(wav_paths, desc='Padding'):
#     y = librosa.load(wav_path, sr=16000)[0]
#     n_frames = len(y)
#     y_sil = np.array([])
#     if n_frames != cutoff_limit:
#         pad_length = cutoff_limit - n_frames

#         with open(wav_path.replace("wav", "phn"), "r") as f:
#             lines = f.readlines()
#         phonemes = np.array([line.split()[-1] for line in lines])
#         for i in np.where(phonemes == "h#")[0]:
#             start_sil, end_sil = list(map(int, lines[i].split()[:-1]))
#             y_sil = np.concatenate([y_sil, y[start_sil:end_sil]])
        
#         if len(y_sil) >= pad_length:
#             y_sil = y_sil[:pad_length]
#         else:
#             extension = pad_length - len(y_sil)
#             y_sil = np.pad(y_sil, (0, extension), mode='wrap')
        
#         np.random.shuffle(y_sil)
#         y = np.concatenate([y, y_sil])
        
#         start, end, phoneme = lines.pop(-1).split()
#         if lines[-1].split()[-1] == "h#":
#             line = " ".join([end, str(cutoff_limit), phoneme])
#         else:
#             line = " ".join([start, str(cutoff_limit), "h#\n"])
#         lines.append(line)

#         with open(wav_path.replace("wav", "phn"), "w") as f:
#             f.writelines(lines)

#         sf.write(wav_path, y, 16000)

#     else:
#         pass

In [6]:
d = pd.read_csv("Datasets\TIMIT-dataset\data.csv")
d

,wav_paths
0,Datasets\TIMIT-dataset\data\DR1\FAKS0\SA1.wav
1,Datasets\TIMIT-dataset\data\DR1\FAKS0\SA2.wav
2,Datasets\TIMIT-dataset\data\DR1\FAKS0\SI1573_1...
3,Datasets\TIMIT-dataset\data\DR1\FAKS0\SI1573_2...
4,Datasets\TIMIT-dataset\data\DR1\FAKS0\SI2203.wav
...,...
6857,Datasets\TIMIT-dataset\data\DR8\MTCS0\SX172.wav
6858,Datasets\TIMIT-dataset\data\DR8\MTCS0\SX262.wav
6859,Datasets\TIMIT-dataset\data\DR8\MTCS0\SX352.wav
6860,Datasets\TIMIT-dataset\data\DR8\MTCS0\SX442.wav


# TFrecords

## Write

In [96]:
def ArgParser():
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--n_splits", dest="n_splits", type=int, default=5)
    parser.add_argument("--sample_rate", dest="sample_rate", type=int, default=16000)
    parser.add_argument("--n_fft", dest="n_fft", type=int, default=2048)
    parser.add_argument("--window_size", dest="window_size", type=int, default=400)
    parser.add_argument("--hop_length", dest="hop_length", type=int, default=160) # 160 samples = 10ms
    parser.add_argument("--n_mels", dest="n_mels", type=int, default=64)
    parser.add_argument("--max_samples", dest="max_samples", type=int, default=70000)
    parser.add_argument("--main_dir", dest='main_dir', type=str, default="Datasets/TIMIT-dataset/tfrec_data")

    
    args = parser.parse_known_args()[0]
    seq_len = int(np.ceil(args.max_samples / args.hop_length))
    parser.add_argument("--seq_len", type=int, default=seq_len)
    return parser.parse_known_args()[0]

args = ArgParser()
args

Namespace(hop_length=160, main_dir='Datasets/TIMIT-dataset/tfrec_data', max_samples=70000, n_fft=2048, n_mels=64, n_splits=5, sample_rate=16000, seq_len=438, window_size=400)

In [82]:
class TFRWriter():
    def __init__(self, args):
        self.samples = d['wav_paths'].tolist()
        self.args = args
        self.fmin = 0
        self.fmax = 8000
        self.top_db = 80
        self.dict_path = "Datasets\TIMIT-dataset\phoneme_dict.json"
        self.phoneme_dict = self.get_dict()


    def _bytes_feature(self, value):
        """Returns a bytes_list from a string / byte."""
        if isinstance(value, type(tf.constant(0))):
            value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


    def serialize_example(self, *args):
        feature = {
            'audio': self._bytes_feature(args[0]),
            'phonemes': self._bytes_feature(args[1]),
            'frames': self._bytes_feature(args[2]),
            'filename': self._bytes_feature(args[3])}

        example_proto = tf.train.Example(
            features=tf.train.Features(feature=feature))
        return example_proto.SerializeToString()


    def get_shards(self):
        speaker_id = [sample.split('\\')[4] for sample in self.samples]
        skf = StratifiedKFold(
            n_splits=self.args.n_splits, shuffle=True, random_state=42)
        return [
            list(map(lambda x: self.samples[x], j)) 
            for i, j in skf.split(self.samples, speaker_id)]


    def get_dict(self):
        # phonemes = set()
        phonemes = set()
        markers = ['h#', 'pau', 'epi']
        for sample in self.samples:
            base_path = os.path.splitext(sample)[0]
            with open(base_path + '.phn', "r") as f:
                for line in f.readlines():
                    phoneme = line.split()[-1]
                    if not phoneme in markers:
                        phonemes.add(phoneme)
        phonemes = markers + sorted(Counter(phonemes), key=Counter(phonemes).get, reverse=True)
        phonemes_dict = {v: i+1 for i, v in enumerate(phonemes)}
        with open(self.dict_path, "w") as f:
            json.dump(phonemes_dict, f, sort_keys=False, indent=4)
        return phonemes_dict
   
    
    def get_shard_data(self, samples, shard):
        for sample in tqdm(
                samples, total=len(samples), desc=f"Writing shard {shard}"):
            base_path = os.path.splitext(sample)[0]
            p_frames, phonemes = [0], []
            with open(base_path + ".phn") as f:
                for line in f.readlines():
                    p_frame, phoneme = line.split()[1::]
                    p_frames.append(int(p_frame) // self.args.hop_length)
                    phonemes.append(str(phoneme))
            phonemes = list(map(self.phoneme_dict.get, phonemes))
            waveform = tf.io.read_file(base_path + ".wav")
            filename = str.encode("/".join(sample.split('\\')[-3::]))
            yield {
                "audio": waveform,
                "phonemes": tf.io.serialize_tensor(phonemes),
                "frames": tf.io.serialize_tensor(p_frames),
                "filename": filename}


    def write(self):
        for shard, samples in enumerate(self.get_shards()):
            with tf.io.TFRecordWriter(
                    f"Datasets/TIMIT-dataset/tfrec_data/train_{shard+1}.tfrec") as f:
                for sample in self.get_shard_data(samples, shard+1):
                    example = self.serialize_example(
                        sample['audio'], sample['phonemes'], 
                        sample['frames'], sample['filename'])
                    f.write(example)

TFRWriter(args).write()

Writing shard 1:   0%|          | 0/1373 [00:00<?, ?it/s]

Writing shard 2:   0%|          | 0/1373 [00:00<?, ?it/s]

Writing shard 3:   0%|          | 0/1372 [00:00<?, ?it/s]

Writing shard 4:   0%|          | 0/1372 [00:00<?, ?it/s]

Writing shard 5:   0%|          | 0/1372 [00:00<?, ?it/s]

## Loading

In [83]:
def decode_audio(string):
    audio = tf.audio.decode_wav(string, desired_samples=args.max_samples)[0]
    return tf.squeeze(audio, axis=-1)

def get_spectrogram(waveform):
    waveform /= 32678
    spectrogram = tf.abs(tfio.audio.spectrogram(
        waveform, nfft=args.n_fft, window=args.window_size, 
        stride=args.hop_length))
    mel_spectrogram = tfio.audio.melscale(
        spectrogram, rate=args.sample_rate, mels=args.n_mels, 
        fmin=0, fmax=8000)
    mel_spectrogram = tfio.audio.dbscale(
        mel_spectrogram, top_db=80)
    return mel_spectrogram

def get_framewise_labels(p_frames, phonemes):
    labels = []
    for i in range(1, len(p_frames)):
        for j in range(p_frames[i-1], p_frames[i]):
            labels.append(phonemes[i-1])
    labels = tf.convert_to_tensor(labels)
    padding = tf.zeros([args.seq_len-labels.shape[0]], dtype=tf.int32)
    return tf.concat([labels, padding], axis=0)

def get_binary_labels(p_frames):
    labels = tf.tensor_scatter_nd_update(
        tensor=tf.zeros([p_frames[-1]+1], dtype=tf.int32), 
        indices=tf.expand_dims(p_frames, axis=1), 
        updates=tf.ones([p_frames.shape[0]], dtype=tf.int32))
    padding = tf.zeros([args.seq_len-labels.shape[0]], dtype=tf.int32)
    return tf.concat([labels, padding], axis=0)

def read_tfrecord(example):
    feature_description = {
        'audio': tf.io.FixedLenFeature([], tf.string),
        'phonemes': tf.io.FixedLenFeature([], tf.string),
        'frames': tf.io.FixedLenFeature([], tf.string),
        'filename': tf.io.FixedLenFeature([], tf.string)}
    
    example = tf.io.parse_single_example(example, feature_description)
    example['audio'] = decode_audio(example['audio'])
    example['frames'] = tf.io.parse_tensor(example['frames'], out_type=tf.int32)
    example['phonemes'] = tf.io.parse_tensor(example['phonemes'], out_type=tf.int32)
    example['binary_labels'] = get_binary_labels(example['frames'])
    example['framewise_labels'] = get_framewise_labels(example['frames'], example['phonemes'])
    example['attention_mask'] = tf.constant(
        [True if i < example['frames'][-1] else False for i in range(args.seq_len)])
    del(example['frames'], example['phonemes'])
    return example

tfr_path = r"Datasets/TIMIT-dataset/tfrec_data/train_1.tfrec"
raw_dataset = tf.data.TFRecordDataset(tfr_path)

for raw_record in raw_dataset.take(1):
    example = read_tfrecord(raw_record)
    print(example['framewise_labels'])

tf.Tensor(
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 27 23 23 23 23 23 23 23 23 23 23 23  4  4
  4  4  4 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 22 22 22 22 22 22
 22 22 22 22  3  3  3  3  3  3  3  3 13 13 13 30 30 30 30 30 30 30 30 30
 55 55 55 55 42 31 31 31 31 31  7  7  7  7 16 16 16 16 16 16 44 44 44 44
 44 44 44 44 44 44 44 35 35 35 35 35 35 35 35 35 35 35 35 30 30 30 30 30
 30 30 30 31 31 31 31 31  4  4  4  4  4  4  4 59 59 59 59 59 59 59 59 59
 59 59 59 59 59 59 59 59 59 41 41 41 41 41 41 41 41 41 30 30 30 30 30 30
 30 30 30 30 35 35 35 35 35 35 35 35 35 35 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 57 57 57 57 57 57 49 49 49 41 41 41 41 41 41 26 26
 26 26 26 26 26 26 26 26  7  7  7  7  7 16 16 16 16 16 54 54 54 54 54 54
 12 12 12 12 12 12 12 12 12 12 12 12 12 

In [111]:
t = tf.zeros([5], dtype=tf.int32)
t

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 0])>

In [115]:
tf.tensor_scatter_nd_add(t, [[t.shape[0]-1]], [1])

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 1])>

In [117]:
class TIMITDataset():
    def __init__(self, args):
        self.files = [os.path.join(args.main_dir, f) for f in os.listdir(args.main_dir)]
        self.args = args
        self.AUTOTUNE = tf.data.experimental.AUTOTUNE
        self.BUFFER_SIZE = 512
        self.BATCH_SIZE = 16
        self.train_files, self.test_files = train_test_split(
            self.files, test_size=0.2, shuffle=True)

    def decode_audio(self, string):
        audio = tf.audio.decode_wav(string, desired_samples=args.max_samples)[0]
        return tf.squeeze(audio, axis=-1)

    def load_dataset(self, files):
        ignore_order = tf.data.Options()
        ignore_order.experimental_deterministic = False
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.with_options(ignore_order)
        dataset = dataset.map(self.read_tfrecord, num_parallel_calls=self.AUTOTUNE)
        return dataset

    def get_spectrogram(self, waveform): # TODO
        waveform /= 32678
        spectrogram = tf.abs(tfio.audio.spectrogram(
            waveform, nfft=self.args.n_fft, window=self.args.window_size, 
            stride=self.args.hop_length))
        mel_spectrogram = tfio.audio.melscale(
            spectrogram, rate=self.args.sample_rate, mels=self.args.n_mels, 
            fmin=0, fmax=8000)
        mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)
        return mel_spectrogram

    def get_binary_labels(self, p_frames): # TODO
        tf.tensor_scatter_nd_add(p_frames, [[-1]], [1])
        print(p_frames)

        labels = tf.tensor_scatter_nd_update(
            tensor=tf.zeros([p_frames[-1]+1], dtype=tf.int32), 
            indices=tf.expand_dims(p_frames, axis=1), 
            updates=tf.ones([p_frames.shape[0]], dtype=tf.int32))
        padding = tf.zeros([self.args.seq_len-len(labels)], dtype=tf.int32)
        return tf.concat([labels, padding], axis=0)

    def get_framewise_labels(self, p_frames, phonemes): # TODO
        labels = []
        for i in range(1, len(p_frames)):
            for j in range(p_frames[i-1], p_frames[i]):
                labels.append(phonemes[i-1])
        labels = tf.convert_to_tensor(labels)
        print(labels)
        # padding = tf.zeros([self.args.seq_len-len(labels)], dtype=tf.int32)
        # return tf.concat([labels, padding], axis=0)
        return

    def read_tfrecord(self, example):
        feature_description = {
            'audio': tf.io.FixedLenFeature([], tf.string),
            'phonemes': tf.io.FixedLenFeature([], tf.string),
            'frames': tf.io.FixedLenFeature([], tf.string),
            'filename': tf.io.FixedLenFeature([], tf.string)}
        
        example = tf.io.parse_single_example(example, feature_description)
        example['audio'] = self.decode_audio(example['audio'])
        example['frames'] = tf.io.parse_tensor(example['frames'], out_type=tf.int32)
        example['phonemes'] = tf.io.parse_tensor(example['phonemes'], out_type=tf.int32)
        example['binary_labels'] = self.get_binary_labels(example['frames'])
        example['framewise_labels'] = self.get_framewise_labels(example['frames'], example['phonemes'])
        example['attention_mask'] = tf.constant(
            [True if i < example['frames'][-1] else False for i in range(self.args.seq_len)])
        del(example['frames'], example['phonemes'])
        return example
 
    def SpecAugment(self, sample):
        waveform = sample['audio']
        spectrogram = tfio.audio.freq_mask(spectrogram, param=10)
        spectrogram = tfio.audio.time_mask(spectrogram, param=10)
        sample['audio'] = spectrogram
        return sample


    def train(self):
        dataset = self.load_dataset(self.train_files)
        dataset = dataset.map(self.SpecAugment, num_parallel_calls=self.AUTOTUNE)
        dataset = dataset.repeat()
        dataset = dataset.shuffle(self.BUFFER_SIZE)
        dataset = dataset.batch(self.BATCH_SIZE)
        dataset = dataset.prefetch(self.AUTOTUNE)
        return dataset


    def test(self):
        dataset = self.load_dataset(self.test_files)
        dataset = dataset.shuffle(self.BUFFER_SIZE)
        dataset = dataset.batch(self.BATCH_SIZE)
        dataset = dataset.cache()
        dataset = dataset.prefetch(self.AUTOTUNE)
        return dataset

train = TIMITDataset(args).train()
train

Tensor("ParseTensor:0", dtype=int32)


TypeError: in user code:

    <ipython-input-108-cd4347262d25>:67 read_tfrecord  *
        example['binary_labels'] = self.get_binary_labels(example['frames'])
    <ipython-input-108-cd4347262d25>:38 get_binary_labels  *
        labels = tf.tensor_scatter_nd_update(
    E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\array_ops.py:3225 ones
        shape = ops.convert_to_tensor(shape, dtype=dtypes.int32)
    E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\profiler\trace.py:163 wrapped
        return func(*args, **kwargs)
    E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\ops.py:1566 convert_to_tensor
        ret = conversion_func(value, dtype=dtype, name=name, as_ref=as_ref)
    E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\constant_op.py:346 _constant_tensor_conversion_func

In [ ]:
import librosa.display

spectrogram = list(next(iter(train)).values())[-1]

plt.figure(figsize=(10, 7))
n_samples = spectrogram.shape[0]
row = 4; col = int(n_samples / 4)

for i in range(n_samples):
    plt.subplot(row, col, i+1)
    plt.axis("off")
    librosa.display.specshow(spectrogram[i].numpy().T)

plt.tight_layout()
plt.show()

In [ ]:
binary = list(next(iter(train)).values())[0]
binary[0]